https://towardsdatascience.com/top-5-geospatial-data-apis-for-advanced-analysis-79349605c86d

In [8]:
import pandas as pd
import os
import re
import folium
from datetime import datetime, timedelta
import requests

In [2]:
from pathlib import Path
path_folder = Path("/Users/keithlowton/Desktop/Ks/Python code/Geospatial data")
print(path_folder.exists())
print(path_folder)

True
/Users/keithlowton/Desktop/Ks/Python code/Geospatial data


#### Functions

In [ ]:
# Function to convert travel time to minutes
def convert_time_to_minutes(time_str):
    hours = re.search(r"(\d+) hour", time_str)
    minutes = re.search(r"(\d+) mins", time_str)

    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))

    return total_minutes


# Function to extract distance in kilometers as float
def extract_distance_km(distance_str):
    return float(re.search(r"([\d\.]+) km", distance_str).group(1))

In [ ]:
def create_folium_map(df):
    # Create a base map
    m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=13)

    # Create a color map for distances
    colormap = branca.colormap.LinearColormap(
        colors=["green", "orange", "red"],
        vmin=df["Travel Time (mins)"].min(),
        vmax=df["Travel Time (mins)"].max(),
        caption="Travel Time to Airport (min)",
    )

    # Add markers to the map using CircleMarker for gradient color
    for index, row in df.iterrows():
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=8,
            color=colormap(row["Travel Time (mins)"]),
            fill=True,
            fill_color=colormap(row["Travel Time (mins)"]),
            fill_opacity=0.8,
            popup=row["Name"],
            tooltip=f"Travel Time: {row['Travel Time (mins)']} min",
        ).add_to(m)

    # Add the color map legend
    m.add_child(colormap)

    return m


#### Pull the data in

In [5]:
df = pd.read_csv(os.path.join(path_folder, "opendatabcn_allotjament_hotels-csv.csv"), encoding = 'utf-16')

# Define a list of column names to select
selected_column_names = ['name', 'addresses_neighborhood_name', 'addresses_district_name', 'geo_epgs_4326_lat', 'geo_epgs_4326_lon']

# Select the specified columns from the DataFrame
df = df[selected_column_names]

# Define a dictionary to map the old column names to the new names
column_name_mapping = {
    'name': 'Name',
    'addresses_neighborhood_name': 'Neighborhood',
    'addresses_district_name': 'District',
    'geo_epgs_4326_lat': 'Latitude',
    'geo_epgs_4326_lon': 'Longitude'
}

# Use the rename method to rename the columns
df.rename(columns=column_name_mapping, inplace=True)

# Extract the name of the hotel
df['Name'] = df['Name'].str.split(' - ').str[0]

# Display the first few rows of the DataFrame
df.head()

,Name,Neighborhood,District,Latitude,Longitude
0,Hotel California,el Barri Gòtic,Ciutat Vella,41.380991,2.175322
1,Hotel Call,el Barri Gòtic,Ciutat Vella,41.382294,2.175713
2,Hotel Cantón,el Barri Gòtic,Ciutat Vella,41.378538,2.178066
3,Hotel The Social Hub Barcelona Poblenou,Provençals del Poblenou,Sant Martí,41.409894,2.204651
4,Hotel Barcelona Apolo,el Poble-sec,Sants-Montjuïc,41.374782,2.170336


In [ ]:
# Define the API key and endpoint
api_key = "YOUR_API_KEY"
endpoint = "https://api.distancematrix.ai/maps/api/distancematrix/json"

# Define the coordinates of the airport
latitude_airport = 41.30423
longitude_airport = 2.07345

# Define the travel mode
mode = "transit"

# Calculate the UNIX timestamp for 8:00 AM the next day
now = datetime.now()
next_morning = datetime(now.year, now.month, now.day) + timedelta(days=1, hours=8)
departure_time = int(next_morning.timestamp())

# Prepare an empty list to store the results
results = []

# Loop through each hotel in the DataFrame
for index, row in df.iterrows():
    origin = f"{row['Latitude']},{row['Longitude']}"
    destination = f"{latitude_airport},{longitude_airport}"

    try:
        # Send the request for each hotel
        response = requests.get(
            endpoint,
            params={
                "origins": origin,
                "destinations": destination,
                "departure_time": departure_time,
                "mode": mode,
                "key": api_key,
            },
        )

        # Check if the response status is OK
        if response.status_code == 200:
            # Parse the response
            data = response.json()

            # Check if the response contains valid data
            if data["rows"][0]["elements"][0]["status"] == "OK":
                distance = data["rows"][0]["elements"][0]["distance"]["text"]
                duration = data["rows"][0]["elements"][0]["duration"]["text"]
            else:
                # Handle case where API response is valid but no data is returned
                distance = None
                duration = None
        else:
            # Handle non-200 response status
            distance = None
            duration = None

    except Exception as e:
        # Handle exceptions, such as connection errors or timeouts
        print(f"Error fetching data for {row['Name']}: {e}")
        distance = None
        duration = None

    # Append the result for this hotel
    results.append({"Name": row["Name"], "Distance to Airport": distance, "Travel Time to Airport": duration})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Merge the results with the original DataFrame
distances_df = pd.merge(df, results_df, on="Name")

# Visualize the first rows of the dataframe
distances_df.head()


In [ ]:
# Apply the functions to create new columns
distances_df["Travel Time (mins)"] = distances_df["Travel Time to Airport"].apply(convert_time_to_minutes)
distances_df["Distance (km)"] = distances_df["Distance to Airport"].apply(extract_distance_km)

# Visualize the first results of the DataFrame
distances_df.head()


In [ ]:
create_folium_map(distances_df)
